In [1]:
pip install bitarray

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.3/278.3 kB 15.1 MB/s eta 0:00:00


In [2]:
pip install mmh3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 4.2 MB/s eta 0:00:00


In [3]:
import math
import mmh3
from bitarray import bitarray

class BloomFilter(object):

    def __init__(self, items_count, fp_prob):
        self.fp_prob = fp_prob
        self.size = self.get_size(items_count, fp_prob)
        self.hash_count = self.get_hash_count(self.size, items_count)
        self.bit_array = bitarray(self.size)
        self.bit_array.setall(0)

    def add(self, item):
        digests = []
        for i in range(self.hash_count):
            digest = mmh3.hash(item, i) % self.size
            digests.append(digest)

        # set the bit True in bit_array
        self.bit_array[digests] = True

    def check(self, item):
        for i in range(self.hash_count):
            digest = mmh3.hash(item, i) % self.size
            if self.bit_array[digest] == False:
                return False
        return True

    @classmethod
    def get_size(self, n, p):
        m = -(n * math.log(p)) / (math.log(2)**2)
        return int(m)

    @classmethod
    def get_hash_count(self, m, n):
        k = (m/n) * math.log(2)
        return int(k)


In [4]:
from random import shuffle

n = 20
p = 0.05

bloomf = BloomFilter(n, p)
print("Size of bit array:{}".format(bloomf.size))
print("False positive Probability:{}".format(bloomf.fp_prob))
print("Number of hash functions:{}".format(bloomf.hash_count))

word_present = ['EKTA', 'VPPCOE', 'abundance', 'abundant', 'accessible',
                'bloom', 'blossom', 'bolster', 'bonny', 'bonus', 'bonuses',
                'coherent', 'cohesive', 'colorful', 'comely', 'comfort',
                'gems', 'generosity', 'generous', 'generously', 'genial']

word_absent = ['Sion', 'cheater', 'hate', 'war', 'humanity',
               'racism', 'hurt', 'nuke', 'gloomy', 'facebook',
               'twitter']

for item in word_present:
    bloomf.add(item)

shuffle(word_present)
shuffle(word_absent)

test_words = word_present[:10] + word_absent
shuffle(test_words)

for word in test_words:
    if bloomf.check(word):
        if word in word_absent:
            print("'{}' is a False Positive!".format(word))
        else:
            print("'{}' is true positive!".format(word))
    else:
        print("'{}' is a True Negative!".format(word))


Size of bit array:124
False positive Probability:0.05
Number of hash functions:4
'Sion' is a True Negative!
'colorful' is true positive!
'generously' is true positive!
'racism' is a True Negative!
'facebook' is a True Negative!
'bloom' is true positive!
'bonuses' is true positive!
'gloomy' is a True Negative!
'comfort' is true positive!
'hate' is a True Negative!
'blossom' is true positive!
'cohesive' is true positive!
'hurt' is a True Negative!
'EKTA' is true positive!
'generous' is true positive!
'nuke' is a True Negative!
'war' is a True Negative!
'humanity' is a False Positive!
'abundance' is true positive!
'cheater' is a True Negative!
'twitter' is a True Negative!
